In [1]:
import os 
import sys
base_dir = os.path.abspath('')
curr_dir = os.path.dirname(base_dir)
if not curr_dir in sys.path: sys.path.append(curr_dir)

In [2]:
from glob import glob
from utils import to_markdown
from urllib.parse import urlparse
from collections import defaultdict

In [3]:
def contains_course_info(url):
    url_prased = urlparse(url)
    return len(url_prased.path.split("/"))>5

In [4]:
files = []
with open("../stevens_scraper/crawled_urls.txt") as fp:
    for line in fp:
        files.append(line.strip())
files = list(map(lambda x: x.lower(), files))
files = list(set(files))
files = list(filter(lambda file: "narrative-courses" not in file, files))
files = list(filter(lambda file: "javascript" not in file, files))
files = list(filter(lambda file: ".pdf" not in file, files))
files = list(filter(lambda file: ".aspx" not in file, files))
files = list(filter(lambda file: "degree-programs2" not in file, files))
files = list(filter(lambda file: "2023-2024" in file, files))
files = list(filter(lambda file: "#middle" not in file, files))
files = list(filter(contains_course_info , files))
print(len(files))

with open("../stevens_scraper/crawled_urls_v1.txt", "w+") as fp:
    fp.write("\n".join(sorted(files, key=len)))

3434


In [5]:
primary = defaultdict(list)
for url in files:
    url_prased = urlparse(url.lower())
    cat = url_prased.path.replace("/en/2023-2024/academic-catalog/", "").split("/")[0]
    primary[cat].append(url)

In [6]:
for key in primary.keys(): print(key, len(primary[key]))

courses 2889
 12
department-of-chemical-engineering-and-materials-science 22
undergraduate-education 77
school-of-business 73
department-of-computer-science 23
school-of-systems-and-enterprises 41
financing-a-stevens-education 44
department-of-chemistry-and-chemical-biology 16
tuition-fees-and-other-expenses-for-undergraduate-students 5
department-of-mathematical-sciences 14
student-services 26
department-of-mechanical-engineering 31
department-of-civil-environmental-and-ocean-engineering 29
graduate-education 24
tuition-fees-and-other-expenses-for-graduate-students 5
school-of-humanities-arts-and-social-sciences 28
department-of-electrical-and-computer-engineering 35
department-of-biomedical-engineering 11
department-of-physics 17
academic-integrity 2
schaefer-school-of-engineering-and-science 4
student-life 6


In [7]:
sorted(primary["courses"])

['https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/500',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/500/aai-551',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/600',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/600/aai-627',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/600/aai-628',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/600/aai-646',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/600/aai-672',
 'https://stevens.smartcatal

In [8]:
def create_course_metadata(url):
    metadata = {}
    path = urlparse(url).path.replace("/en/2023-2024/academic-catalog/courses/", "")
    paths = path.split("/")
    metadata["course_acronym"] = paths[0].split("-")[0].upper()
    metadata["course_name"] = " ".join(paths[0].split("-")[1:]).title()
    if len(paths)>1:
        try:
            metadata["course_number"] = int(paths[-1].split("-")[-1])
        except ValueError:
            pass
    return metadata

In [9]:
from bs4 import BeautifulSoup
def extract_course_credits(html):
    soup = BeautifulSoup(html, 'html.parser')
    if selector := soup.select_one("div.credits"):
        try:
            return float(selector.text.strip().split("-")[-1])
        except ValueError:
            pass
    return None

In [10]:
create_course_metadata(primary["courses"][44])

{'course_acronym': 'HAR', 'course_name': 'Humanitiesart', 'course_number': 471}

In [11]:
# with open("../stevens_scraper/courses.txt", "w+") as fp:
#     fp.write("\n".join(primary["courses"]))

In [12]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
df = pd.read_csv("../data/courses.csv")

In [13]:
assert df.shape[0]==df["url"].str.endswith("/").sum()

In [14]:
df["url"] = df["url"].apply(lambda x: x[:-1])

In [15]:
df["course_credits"] = df["html"].progress_apply(extract_course_credits)

100%|██████████| 2889/2889 [00:32<00:00, 88.49it/s]


In [16]:
df["course_credits"].unique()

array([ 3. ,  2. , 60. ,  4. ,  6. ,  nan, 15. , 10. ,  0. ,  1. ,  1.5,
        0.5,  0.7,  0.6,  9. , 12. ,  5. ])

In [17]:
url_info = []
for _, row in df.iterrows():
    url_meta = create_course_metadata(row["url"])
    if course_credits:=row["course_credits"]:
        url_meta["course_credits"] = course_credits
    url_meta.update({"page_content": row["text"]})
    url_info.append(url_meta)

In [18]:
url_info[50]

{'course_acronym': 'CPE',
 'course_name': 'Computer Engineering',
 'course_number': 590,
 'course_credits': 3.0,
 'page_content': '\n\nCPE 590 Algorithms\r\n\r\n\tThis is a course on more complex data structures, and algorithm design and analysis, using one or more modern imperative language(s), as chosen by the instructor. Topics include: advanced and/or balanced search trees; hashing; further asymptotic complexity analysis; standard algorithm design techniques; graph algorithms; complex sort algorithms; and other "classic" algorithms that serve as examples of design techniques.\r\n\n\r\n\t\tCredits\r\n\t\r\n\t\t3\r\n\t\n\r\n\tCross Listed Courses\r\nCS 590\n\r\n\t\tPrerequisite\r\n\tCS 570 and a Graduate Student\r\n\nDistributionComputer Engineering Program\r\n\r\nComputer Science Program\n\r\n\t\tTypically Offered Periods\r\n\tFall Semester\n\nSpring Semester\n\nSummer Session 1\n\n'}

In [19]:
# for url in url_meta:
#     with open("stevens_scraper/department_urls.txt", "a") as fp:
#         fp.write(url+"\n")

In [40]:
data = pd.DataFrame.from_records(url_info) 
data.to_csv("../data/courses_info_w_meta.csv", index=False)

In [45]:
",".join(data["course_acronym"].unique())

'SYS,PME,EE,HSS,MA,CM,DS,FE,CHE,NE,CH,HAR,ACC,HPL,MT,CPE,CE,CAL,ENGR,SSW,HLI,PEP,FIN,COMM,OE,HST,EM,CS,BIA,ME,FA,NIS,BIOE,TG,LFR,HMU,EN,ELC,MGT,HHS,MIS,SM,BT,AAI,SES,BIO,BME,CLK,TM,TE,IPD,LTL,DE,QF,LSP,NANO,ISE,EMT,GEN,IDE,PIN,ES,ECON,HTH,LCH,SEF,HONR,SOC,HUM,PAE'

In [21]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.retrievers.self_query.chroma import ChromaTranslator

In [22]:
model_kwargs = {
    "trust_remote_code": True,
    # "device": "cpu"
    }
encode_kwargs={"normalize_embeddings": True}
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

In [23]:
docs = [
    Document(
        page_content=doc["page_content"],
        metadata=dict((key, value) for key, value in doc.items() if key!="page_content")
    )
    for doc in url_info
]
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)
# vectorstore = Chroma.from_documents(docs, embeddings, collection_name="course_info", persist_directory="../data/chromadb")
vectorstore = Chroma(collection_name="course_info", persist_directory="../data/chromadb", embedding_function=embeddings)

In [24]:
vectorstore._collection.count()

2889

In [25]:
vectorstore.persist()

c:\Users\smend\miniconda3\envs\stevens\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [26]:
# Ensure to only use the contain comparator while querying this field.

In [46]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="course_name",
        description="""
        The name of the Course
        Sample course name are Computer Engineering and Machine Learning
        Ensure to do a case insensitive match
        """.strip(),
        type="string",
    ),
    AttributeInfo(
        name="course_acronym",
        description="""
        The acronym of the Course[SYS,PME,EE,HSS,MA,CM,DS,FE,CHE,NE,CH,HAR,ACC,HPL,MT,CPE,CE,CAL,ENGR,SSW,HLI,PEP,FIN,COMM,OE,HST,EM,CS,BIA,ME,FA,NIS,BIOE,TG,LFR,HMU,EN,ELC,MGT,HHS,MIS,SM,BT,AAI,SES,BIO,BME,CLK,TM,TE,IPD,LTL,DE,QF,LSP,NANO,ISE,EMT,GEN,IDE,PIN,ES,ECON,HTH,LCH,SEF,HONR,SOC,HUM,PAE]
        """.strip(),
        type="string",
    ),
    AttributeInfo(
        name="course_number",
        description="The number for that particular course",
        type="integer",
    ),
    AttributeInfo(
        name="course_credits",
        description="""
        The number of credits obtained on completion of the course
        """.strip(),
        type="float",
    ),
]
document_content_description = "Information regarding the course"
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    structured_query_translator=ChromaTranslator(),
    # enable_limit=True
)

In [51]:
question = """

```What is the course BIA 660 about?```
Sample filters are:
and(eq("course_acronym", "SYS"),  eq("course_number", 100))
or(eq("course_acronym", "NIS"), eq("course_number", 678))
""".strip()

In [52]:
retriever_prompt = f"""{question}""".strip()

In [53]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)
prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
)

output_parser = StructuredQueryOutputParser.from_components()
query_constructor = prompt | llm | output_parser

print(prompt.format(query=retriever_prompt))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

In [54]:
print(query_constructor.invoke(question))

query='BIA 660' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='course_acronym', value='BIA'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='course_number', value=660)]) limit=None


In [55]:
context = retriever.invoke(retriever_prompt)
context

[Document(page_content='\n\nBIA 660 Web Mining\r\n\r\n\tIn this course, students will learn through hands-on experience how to extract data from the web and analyze web-scale data using distributed computing. Students will learn different analysis methods that are widely used across the range of internet companies, from start-ups to online giants like Amazon or Google. At the end of the course, students will apply these methods to answer real scientific question or to create a useful web application.\r\n\n\r\n\t\tCredits\r\n\t\r\n\t\t3\r\n\t\n\n\nDistributionSchool of Business\n\r\n\t\tTypically Offered Periods\r\n\tFall Semester\n\nSpring Semester\n\nSummer Session 1\n\n', metadata={'course_acronym': 'BIA', 'course_credits': 3.0, 'course_name': 'Business Intelligence And Analytics', 'course_number': 660})]

In [60]:
# context[0].metadata

In [61]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [   
        ("system", "You are an assistant for question-answering tasks related to Stevens Institute Of Technology."),
        ("human", """ 
        Use the following pieces of retrieved context to answer the question.
        If you don't know the answer, just say that you don't know.
        If the topic is related to a course then ensure to mention to course numbers.
        Question: {question}
        Context: {context}
        Answer:"""),
    ]
)
chat_template

ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an assistant for question-answering tasks related to Stevens Institute Of Technology.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template=" \n        Use the following pieces of retrieved context to answer the question.\n        If you don't know the answer, just say that you don't know.\n        If the topic is related to a course then ensure to mention to course numbers.\n        Question: {question}\n        Context: {context}\n        Answer:"))])

In [62]:
combine_docs_chain = create_stuff_documents_chain(llm, chat_template)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [63]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | SelfQueryRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001DE881B5450>, query_constructor=RunnableBinding(bound=FewShotPromptTemplate(input_variables=['query'], examples=[{'i': 1, 'data_source': '```json\n{{\n    "content": "Lyrics of a song",\n    "attributes": {{\n        "artist": {{\n            "type": "string",\n            "description": "Name of the song artist"\n        }},\n        "length": {{\n            "type": "integer",\n            "description": "Length of the song in seconds"\n        }},\n        "genre": {{\n            "type": "string",\n            "description": "The song genre, one of "pop", "rock" or "rap""\n        }}\n    }}\n}}\n```', 'user_query': 'What are songs by Taylor Swift or Katy Perry about teenage romance under 3 minutes long in the dance pop genre', 'structured_request': '```json\n{

In [64]:
response = retrieval_chain.invoke({"input": retriever_prompt, "question": question})
to_markdown(response["answer"])

> Based on the provided context, someone interested in finance and marketing can take the following courses at Stevens Institute of Technology:
> 
> 1. FIN 684 Financial Services Industry Marketing and Sales - This course focuses on selling and marketing IT strategies in the financial services industry, covering topics such as the "sell-side," the "buy-side," the selling distribution process, e-business selling strategies, marketing strategies, corporate branding, data warehousing, sales data mining, and client partnerships. It is worth 3 credits.
> 
> 2. MGT 641 Marketing Management - This course covers marketing principles from conceptual, analytical, and managerial perspectives, including strategic planning, market segmentation, product life-cycle, new product development, advertising and selling, pricing, distribution, and the impact of governmental and other environmental factors on markets and business structure. It is also worth 3 credits and is typically offered in the Fall Semester, Spring Semester, and Summer Session 1.
> 
> Both courses are distributed through the School of Business.

In [65]:
vectorstore.delete_collection()

PermissionError: [WinError 5] Access is denied: '../data/chromadb\\e59dfdc2-5cb6-4688-9e40-9b6a3e60e154'

In [74]:
for key in primary.keys(): 
    if key not in school_department_keys:
        print(key, len(primary[key])) 

courses 2889
undergraduate-education 77
financing-a-stevens-education 44
student-services 26
 12
tuition-fees-and-other-expenses-for-undergraduate-students 5
graduate-education 24
school-of-humanities-arts-and-social-sciences 28
student-life 6
academic-integrity 2
tuition-fees-and-other-expenses-for-graduate-students 5
